# Cans Notebook for Understanding Transformers

## Purpose
Predict next token in a sequence

In [1]:
# Jargon dictionary
jargon = {
    'vector':
    'multidimensional variable, column of shape 1 x N',

    'tensor':
    'multidimensional variable, higher number of axes than variable, e.g. L x M x N'
}

In [5]:
jargon['tensor']

'multidimensional variable, higher number of axes than variable, e.g. L x M x N'

Position = time
In attention layer, we call the output of the softmax attention. This corresponds to activation, Activation = attention

## Ideas for Interpretability

Multiheadedness: higher chance fo find a good parameter value?

## General structure

Outside the model:

Tokenizer: Input data to distinct tokens
___

1. Embedding: Token to vector
2. Residual Stream with N Blocks of
   - LayerNorm
   1. Multiheaded Attention (multiple attention layers in parallel) 
   - LayerNorm
   2. MLP
   - LayerNorm
3. Unembedding

___
'untokenizer?'

## 1. Embedding

Takes batch of tokens: [batch, position]. Each token corresponds to an it in the range [0, d_vocab]

Define a matrix W_E: [d_vocab, d_model]

W_E maps token to d_model space. Thats why they call it a lookup table

`embedded = W_E[tokens, :]`

### 1.1 Positional Embedding
Maps the position of the token to d_model space.

add this pos embed to d_model space

# 2. LayerNorm

has weights (init = 1)
and biases (init = 0)

input : residual

- mean=0 translation
  tranlated = residual - mean(residual)

- stddev = Root mean square
  stddev = sqrt(mean(residual^2))

normalized = (residual - mean) / variance

weighted = residual @ weights + biases

return weight



# 3. Attention

Attention solves data dependent information exchange.

Every token in sequence


Three linear maps to lower dimensional ```d_head``` space:
- Query: What am I looking for?
- Key: What do I contain?
- Value: if you choose to aggregate this position, you'll actually get the transformed version instead of x. 
If you find me interesting, here's what ill communicate to you

Dot product Q K: How much does the query fit to the key?

Mask to only allow Q K pairs where Q doesn't look into the future: Index Q >= Index K, Lower triangular matrix including qk

divide by sqrt(d_head) -> activation_scores

Apply softmax per query -> attentions (=activations)

multiply by key values

map values with W_O back to the residual stream.

# 4. MLP

Simple feed forward network with two layers

d_mlp usually 4 * d_model

we apply GELU instead of ReLU

# 5. Transformer block

Wrap Attention and Mlp with Layernorms to single block

RESID_PRE -> LayerNorm -> Attention -> RESID_MID -> LayerNorm -> Mlp -> RESID_OUT

# 6. Unembed

Map residual stream to logits in vocab space
return logits 

# A. Compute loss

## A1. Compute cross entropy loss

You create predictions for every token to be able to learn from every sub sentence in the sequence.

compute log_probs = logits.log_softmax(dim=-1) over each d_vocab

## Interpretation of loss in maximum likelyhood estimation

# B. Training